In [ ]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
data.shape

In [ ]:
data.tail()

In [ ]:
data.describe()

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

data['CPM'] = data.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)

In [ ]:
mask_train = data['date'] < '2019-06-22'
data_train = data[mask_train]
data_test = data[~mask_train]

data_train = data_train[data_train['CPM'] >= 0]
data_train = data_train[data_train['CPM'] <= data_train['CPM'].quantile(.95)]

data_test = data_test[data_test['CPM'] >= 0]
data_test = data_test[data_test['CPM'] <= data_test['CPM'].quantile(.95)]

In [ ]:
columns_to_drop = ['date', 'order_id' , 'line_item_type_id', 'integration_type_id', 'revenue_share_percent', 'total_revenue', 'measurable_impressions']
data_train.drop(columns_to_drop, axis = 1, inplace=True)
data_test.drop(columns_to_drop, axis = 1, inplace=True)

In [ ]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=8,
    gamma=0,
    max_depth=10,
    learning_rate=0.1,
    random_state=1567,
)

In [ ]:
regressor.fit(data_train.drop('CPM', axis=1, inplace = False), data_train['CPM'])

In [ ]:
pd.DataFrame(regressor.feature_importances_.reshape(1, -1), columns=['site_id','ad_type_id','geo_id','device_category_id','advertiser_id', 'os_id', 'monetization_channel_id', 'ad_unit_id', 'total_impressions', 'viewable_impressions'])

In [ ]:
pred_CPM = regressor.predict(data_test.drop('CPM', axis=1, inplace = False))

In [ ]:
mean_squared_error(pred_CPM, data_test['CPM'])